In [1]:
'''1. Remove foreign records to a 'foreign dataframe' and delete them from this df. - applicable for  address
2.Find whether all phone numbers are empty, if empty drop those rows (8708)
3.Column Phone matches the Phone number in preferred phone number
Transfer records with non-null values in Phone to a new dataframe with columns EID and phone
Find the count of records (124000)
4. Null values in Phone: (36000 records) Create a new df:Check the content in preferred phone 
and grasp the preferred phone and add it to df with columns eid and phone

'''

import sys
import pandas as pd
from fuzzywuzzy import fuzz

df_txexAddr = pd.read_csv('TxExAddr.csv')
print("original count ",len(df_txexAddr))

df_Addr = df_txexAddr.dropna(how='all', subset=['Street'])
print("Count records after dropping records with no address : ",len(df_Addr))
#print(df_Addr.head(3))

df_Addr = df_Addr[['EID', 'Street']]
df_Addr['EID'] = df_Addr['EID'].str.upper()
print(len(df_Addr))
print(df_Addr.head(3))

original count  166745
Count records after dropping records with no address :  158723
        EID                   Street     City     State         Zip  \
0    evr253  1715 gosnell rd apt 104   Vienna  Virginia       22182   
1     cen95        1808 Intervail Dr   Austin        TX       78746   
2  goldenr1   5735 Diamond Point Cir  El Paso        TX  79912-4160   

         Country Primary Address Type TE_Current_Address_Updated  \
0  United Statea                 Home             9/5/2019 23:01   
1  United States                 Home            2/25/2020 11:37   
2  United States                 Home             7/2/2020 10:09   

                                        Other Street Other City  \
0                                                NaN        NaN   
1             Westlake High School\n4100 Westbank Dr     Austin   
2  Western Hills Elementary School 530 Thunderbir...    El Paso   

  Other State/Province Other Zip/Postal Code Other Country  \
0                  NaN   

C:\Users\le5752\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_addr_csv = df_Addr[50000:100000]
print(len(df_addr_csv))
print(df_addr_csv.head(3))
print(df_addr_csv.tail(3))

50000
            EID                 Street
50544  0499FJZD  7807 Moonflower Drive
50545    RRW372        5929 Merritt Pl
50546   GIRDYGO     2041 Snyder Avenue
             EID                                 Street
104693  0926JGMY                  6019 San Paublo Court
104694    NAJOLI  2611 Bens Branch Drive Apartment 2414
104695    CVC464                      455 Kayton Avenue


In [5]:
raw_eid = df_addr_csv['EID'].dropna()
eid = raw_eid.tolist()
#print(len(eid))
# How many elements each list should have 
n = 1000
pid = {}
for i in range((len(eid) + n - 1) // n ):
    #globals()["eid" + str(i)] = eid[i * n:(i + 1) * n]
    pid[i] = eid[i * n:(i + 1) * n]

In [6]:
import cx_Oracle
connection = cx_Oracle.connect("vipapp/ZJf3__6qjRt8@exa1-scan.austin.utexas.edu:1525/viprp_pdb.austin.utexas.edu")
cursor = connection.cursor()
df_addr_db = pd.DataFrame(columns = ['EID','Address1','Address2','City','State','Zip Code'])
for i in range(50):
    cursor.execute("select UT_EID, vi_addr_line1, vi_addr_line2, vi_city, vi_state,vi_zip from vi_vips_main m, vi_vips_addresses a  where m.ALUMNI_ID = a.ALUMNI_ID AND m.ut_eid IN %s" % str(tuple(pid[i])).replace(',)',')'))
    results = cursor.fetchall()
    df1 = pd.DataFrame(results, columns = ['EID','Address1','Address2','City','State','Zip Code'])
    df_addr_db = df_addr_db.append(df1,ignore_index=True)
cursor.close()
connection.close()                   

In [7]:
print(len(df_addr_db))
df_addr_db = df_addr_db.drop_duplicates()
print(len(df_addr_db))
print(df_addr_db.head(3))

63409
63263
        EID                      Address1     Address2        City State  \
0  0229QPAH  12912 Hill Country Boulevard  Suite F-240      Austin    TX   
1  0243MRGU          3208 Pond View Drive         None  Richardson    TX   
2  0249JGRZ                 13033 FM 2769         None      Austin    TX   

      Zip Code  
0  78738649960  
1  75082245408  
2  78726101633  


In [8]:
addr_diff = pd.DataFrame()
for i in range(len(df_addr_csv)):
#for i in (11,44,55,66,123,133,143,333,445,667,246,988):
    flag = 0
    id_csv = df_addr_csv.iloc[i,0]
    csv_addr = df_addr_csv.iloc[i,1]
    cmp_dbaddr = df_addr_db.loc[df_addr_db['EID'] == id_csv]
    #print(cmp_dbaddr)
    #print(df_addr_csv.loc[df_addr_csv['EID'] == id_csv])
    #print("---")
    
    for j in range(len(cmp_dbaddr)):
        db_addr = cmp_dbaddr.iloc[j,1]
        Token_Set_Ratio = fuzz.token_set_ratio(csv_addr,db_addr)
        if (Token_Set_Ratio > 77):
            flag = 0
            break
        else:
            flag = 1
    
    #print("flag = ", flag)
    
    if (flag == 1):
        addrlist = []
        for k in range(len(cmp_dbaddr)):
            temp = (cmp_dbaddr.loc[cmp_dbaddr['EID'] == id_csv]).iloc[:,1]
            addrlist = temp.values.tolist()
        addr_diff = addr_diff.append({'EID':id_csv, 'DB_Address':addrlist,'CSV_Address':csv_addr}, ignore_index=True)
#print("----------------------------------------------------------------")
#print(addr_diff)
#print("----------------------------------------------------------------")

In [9]:
print(addr_diff.head(11))
print(len(addr_diff))

                             CSV_Address  \
0                        5929 Merritt Pl   
1          9 Kimball Court Apartment 205   
2                  945 N Marion St Apt 3   
3   401 Teravista Pkwy     Apartment 426   
4                       504 Pheasant Rdg   
5                1904 Woodland Ave Apt A   
6                       5315 Dora Street   
7                      107 OMeara circle   
8                       4690 Darlene Way   
9                           5141 Hartson   
10                       1203 S. Gumwood   

                                           DB_Address       EID  
0   [1313 Old Chain Bridge Road, 3105 Cambridge Dr...    RRW372  
1   [13131 West Little York Road, 197 Washington S...     PMR68  
2                                 [922 Old Lake Road]   SMS3977  
3   [10926 Jollyville Road, 1311 Round Rock Avenue...  EGARDNER  
4      [1311 Round Rock Avenue, 2439 Cloud Peak Lane]    RFM285  
5                [PSC 3, 2317 S Pleasant Valley Road]     LDP96  
6        

In [10]:
addr_diff.to_csv(r'C:\Users\le5752\Documents\DB_Python\addr2.csv', index = False)